In [2]:
import numpy as np
import pandas as pd
import gymnasium as gym
from gymnasium import spaces
from copy import deepcopy

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env


In [4]:
tasks = pd.read_csv(r"complexity_by_hour.csv")
tasks['task_complexity'] = tasks['task_complexity'] / 60
tasks = tasks.to_dict(orient='records')

N_total_nodes = 250

In [28]:
class TaskSchedulingEnv(gym.Env):
    def __init__(self, N_total_nodes, total_tasks):
        super(TaskSchedulingEnv, self).__init__()

        self.waiting_capacity = 100  # Maximum waiting tasks
        self.N_total_nodes = N_total_nodes
        self.total_tasks = deepcopy(total_tasks)

        self.upcoming_tasks = self.total_tasks.copy()
        self.waiting_tasks = []
        self.executing_tasks = []
        self.available_nodes = self.N_total_nodes
        self.current_time = self.total_tasks[0]['hourly_time']

        # Observation space: Dict with state and action mask
        self.observation_space = spaces.Box(
                low=0, high=np.inf, shape=(1 + self.waiting_capacity,), dtype=np.float32
            )
        # Action space: Discrete (number of nodes + 1 for action 0)
        self.action_space = spaces.Discrete(self.N_total_nodes + 1)

        self.state = np.array([self.available_nodes] + [0] * self.waiting_capacity, dtype=np.float32)

    def reset(self):
        self.upcoming_tasks = self.total_tasks.copy()
        self.waiting_tasks = []
        self.executing_tasks = []
        self.available_nodes = self.N_total_nodes
        self.current_time = self.total_tasks[0]['hourly_time']

        padded_waiting_times = [0] * self.waiting_capacity
        state = np.array([self.available_nodes] + padded_waiting_times, dtype=np.float32)

        return state, {}

    def step(self, action):
        # Validate the action
        if action < 0 or action > self.available_nodes:
            return self.state, -100, False, False, {}

        # Add new task to waiting list
        if self.upcoming_tasks and self.current_time == self.upcoming_tasks[0]['hourly_time']:
            self.upcoming_tasks[0]['waiting_time'] = 0
            self.waiting_tasks.append(self.upcoming_tasks.pop(0))

        # Update executing tasks
        for task in self.executing_tasks:
            task['complexity'] -= task['nodes_alloc']
            if task['complexity'] <= 0:
                self.available_nodes += task['nodes_alloc']

        self.executing_tasks = [task for task in self.executing_tasks if task['complexity'] > 0]

        # Allocate nodes to the first waiting task
        if self.waiting_tasks and action > 0:
            self.waiting_tasks[0]['nodes_alloc'] = action
            self.executing_tasks.append(self.waiting_tasks.pop(0))
            self.available_nodes -= action

        # Update waiting times
        for task in self.waiting_tasks:
            task['waiting_time'] += 1

        # Compute next state
        waiting_times = [task['waiting_time'] for task in self.waiting_tasks]
        padded_waiting_times = waiting_times + [0] * (self.waiting_capacity - len(waiting_times))
        state = np.array([self.available_nodes] + padded_waiting_times, dtype=np.float32)

        # Compute reward
        reward = -np.sum(waiting_times)

        # Check termination
        done = len(self.upcoming_tasks) == 0 and len(self.waiting_tasks) == 0 and len(self.executing_tasks) == 0

        self.current_time += 1
        action_mask = self._get_action_mask()
        return state, reward, done, False, {"action_mask": action_mask}

    def _get_action_mask(self):
        # Generate a binary mask for valid actions
        mask = np.zeros(self.N_total_nodes + 1, dtype=np.int32)
        for i in range(self.available_nodes + 1):
            mask[i] = 1
        return mask

In [ ]:
tasks = [{'hourly_time': 0, 'complexity': 5}, {'hourly_time': 1, 'complexity': 3}]
env = TaskSchedulingEnv(N_total_nodes=5, total_tasks=tasks)

# Test reset
obs = env.reset()
print("Initial observation:", obs)

In [33]:
class ActionMaskWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)

    def step(self, action):
        return self.env.step(action)

    def reset(self, **kwargs):
        return self.env.reset()

# Initialize the environment
tasks = tasks[:10]
env = TaskSchedulingEnv(N_total_nodes=5, total_tasks=tasks)
wrapped_env = ActionMaskWrapper(env)

# Train the PPO agent
model = PPO("MlpPolicy", wrapped_env, verbose=1)
model.learn(total_timesteps=100)

# Test the agent
obs, _ = wrapped_env.reset()
done = False
while not done:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = wrapped_env.step(action)
    print(f"Action: {action}, Reward: {reward}")

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


c:\Users\DELL\anaconda3\envs\torch_gpu\Lib\site-packages\stable_baselines3\common\on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 6.16     |
|    ep_rew_mean     | -247     |
| time/              |          |
|    fps             | 466      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 2048     |
---------------------------------
Action: 0, Reward: -1
Action: 0, Reward: -3
Action: 1, Reward: -2
Action: 1, Reward: -0.0
Action: 0, Reward: -0.0
